In [1]:
from mujoco_py import load_model_from_path, MjSim, MjViewer
from mujoco_py import MjRenderContextOffscreen
import mujoco_py

from IPython.display import HTML, display
from matplotlib import pyplot as plt
from copy import deepcopy
import numpy as np
import imageio
import base64
import types
import time
import glob
import cv2
import io

In [2]:
def reset_mocap_welds(sim):
    """Resets the mocap welds that we use for actuation.
    """
    if sim.model.nmocap > 0 and sim.model.eq_data is not None:
        for i in range(sim.model.eq_data.shape[0]):
            if sim.model.eq_type[i] == mujoco_py.const.EQ_WELD:
                sim.model.eq_data[i, :] = np.array(
                    [0., 0., 0., 1., 0., 0., 0.])
    sim.forward()
    
def save_video(frames, filename='video/mujoco.mp4', fps=60):
    writer = imageio.get_writer(filename, fps=fps)
    for f in frames:
        writer.append_data(f)
    writer.close()
    
def show_video(filname='video/mujoco.mp4'):
    mp4list = glob.glob(filname)
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay 
            loop controls style="height: 400px;">
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
            </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")
        
def show_image(img):
    #cv2.imshow("test", img)
    plt.figure(figsize = (16,9))
    plt.axis('off')
    plt.imshow(img)

In [3]:
model = load_model_from_path('make_urdf/ur5_robotiq.xml')
n_substeps = 1 #20
sim = MjSim(model, nsubsteps=n_substeps)
viewer = MjViewer(sim)

Creating window glfw


In [4]:
arm_joint_list = ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint', 'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
gripper_joint_list = ['left_finger_joint', 'right_finger_joint']

str = ''
for joint_name in gripper_joint_list:
    str += "{} : {:.3f}, ".format(joint_name, sim.data.get_joint_qpos(joint_name))
print(str)

left_finger_joint : 0.000, right_finger_joint : 0.000, 


In [5]:
init_arm_pos = np.array([-np.pi/2, -np.pi/2, -np.pi/2, -np.pi/2, np.pi/2, 0.0])
for joint_idx, joint_name in enumerate(arm_joint_list):
    sim.data.set_joint_qpos(joint_name, init_arm_pos[joint_idx])
    sim.data.set_joint_qvel(joint_name, 0.0)
reset_mocap_welds(sim)
    
# Move end effector into position.
gripper_target = np.array([0.0, 0.0, -0.1]) + sim.data.get_body_xpos('wrist_3_link')
gripper_rotation = np.array([1., 0., 1., 0.])
sim.data.set_mocap_pos('robot0:mocap', gripper_target)
sim.data.set_mocap_quat('robot0:mocap', gripper_rotation)
for _ in range(10):
    sim.step()

In [6]:
lookat_refer = sim.data.get_body_xpos('target_body_1')
viewer.cam.lookat[0] = lookat_refer[0]
viewer.cam.lookat[1] = lookat_refer[1]
viewer.cam.lookat[2] = lookat_refer[2]
viewer.cam.azimuth = -75
viewer.cam.elevation = -15
viewer.cam.distance = 1.5

In [9]:
gripper_ctrl = -0.05

frames = []
cnt = 0
steps = 300
for j in range(steps):
    sim.data.mocap_pos[0] = sim.data.get_body_xpos('target_body_1') + np.array([0, 0, 0.3])
    sim.data.mocap_quat[0] = np.array([0, 1, 0, 0])

    for gripper_idx in range(len(gripper_joint_list)):
        cur_pos = sim.data.get_joint_qpos(gripper_joint_list[gripper_idx])
        sim.data.ctrl[gripper_idx] = cur_pos + gripper_ctrl
    
    sim.step()
    viewer.render()
#     time.sleep(0.001)

    cnt += sim.model.opt.timestep/viewer._time_per_render
    while cnt > 0:
        cnt -= 1
        img = viewer._read_pixels_as_in_window()
        frames.append(img)
        
#######################################################
#######################################################

gripper_ctrl = -0.05
for j in range(steps):
    sim.data.mocap_pos[0] = sim.data.get_body_xpos('target_body_1') + np.array([0, 0, 0.15])
    sim.data.mocap_quat[0] = np.array([0, 1, 0, 0])

    for gripper_idx in range(len(gripper_joint_list)):
        cur_pos = sim.data.get_joint_qpos(gripper_joint_list[gripper_idx])
        sim.data.ctrl[gripper_idx] = cur_pos + gripper_ctrl

    sim.step()
    viewer.render()
    
    cnt += sim.model.opt.timestep/viewer._time_per_render
    while cnt > 0:
        cnt -= 1
        img = viewer._read_pixels_as_in_window()
        frames.append(img)

#######################################################
#######################################################

gripper_ctrl = 0.05
init_mocap_pos = deepcopy(sim.data.mocap_pos[0])

for j in range(steps):
    sim.data.mocap_pos[0] = init_mocap_pos
    sim.data.mocap_quat[0] = np.array([0, 1, 0, 0])

    for gripper_idx in range(len(gripper_joint_list)):
        cur_pos = sim.data.get_joint_qpos(gripper_joint_list[gripper_idx])
        sim.data.ctrl[gripper_idx] = cur_pos + gripper_ctrl
        #sim.data.ctrl[gripper_idx] = gripper_ctrl

    sim.step()
    viewer.render()
    
    cnt += sim.model.opt.timestep/viewer._time_per_render
    while cnt > 0:
        cnt -= 1
        img = viewer._read_pixels_as_in_window()
        frames.append(img)

#######################################################
#######################################################

gripper_ctrl = 0.05
init_mocap_pos = deepcopy(sim.data.mocap_pos[0])

for j in range(steps):
    sim.data.mocap_pos[0] = init_mocap_pos + np.array([0, 0, 0.001*j])
    sim.data.mocap_quat[0] = np.array([0, 1, 0, 0])

    for gripper_idx in range(len(gripper_joint_list)):
        cur_pos = sim.data.get_joint_qpos(gripper_joint_list[gripper_idx])
        sim.data.ctrl[gripper_idx] = cur_pos + gripper_ctrl

    sim.step()
    viewer.render()

    cnt += sim.model.opt.timestep/viewer._time_per_render
    while cnt > 0:
        cnt -= 1
        img = viewer._read_pixels_as_in_window()
        frames.append(img)

#######################################################
#######################################################

gripper_ctrl = 0.05
init_mocap_pos = deepcopy(sim.data.mocap_pos[0])

for j in range(steps):
    sim.data.mocap_pos[0] = init_mocap_pos + np.array([0.0007*j, 0, 0])
    sim.data.mocap_quat[0] = np.array([0, 1, 0, 0])

    for gripper_idx in range(len(gripper_joint_list)):
        cur_pos = sim.data.get_joint_qpos(gripper_joint_list[gripper_idx])
        sim.data.ctrl[gripper_idx] = cur_pos + gripper_ctrl

    sim.step()
    viewer.render()

    cnt += sim.model.opt.timestep/viewer._time_per_render
    while cnt > 0:
        cnt -= 1
        img = viewer._read_pixels_as_in_window()
        frames.append(img)

#######################################################
#######################################################

gripper_ctrl = 0.05
init_mocap_pos = deepcopy(sim.data.mocap_pos[0])

for j in range(steps):
    init_mocap_pos[2] = max(init_mocap_pos[2] - j*0.0003, 1.1)
    sim.data.mocap_pos[0] = init_mocap_pos
    sim.data.mocap_quat[0] = np.array([0, 1, 0, 0])

    for gripper_idx in range(len(gripper_joint_list)):
        cur_pos = sim.data.get_joint_qpos(gripper_joint_list[gripper_idx])
        sim.data.ctrl[gripper_idx] = cur_pos + gripper_ctrl

    sim.step()
    viewer.render()

    cnt += sim.model.opt.timestep/viewer._time_per_render
    while cnt > 0:
        cnt -= 1
        img = viewer._read_pixels_as_in_window()
        frames.append(img)

#######################################################
#######################################################

gripper_ctrl = -0.005
init_mocap_pos = deepcopy(sim.data.mocap_pos[0])

for j in range(steps):
    sim.data.mocap_pos[0] = init_mocap_pos
    sim.data.mocap_quat[0] = np.array([0, 1, 0, 0])

    for gripper_idx in range(len(gripper_joint_list)):
        cur_pos = sim.data.get_joint_qpos(gripper_joint_list[gripper_idx])
        sim.data.ctrl[gripper_idx] = cur_pos + gripper_ctrl

    sim.step()
    viewer.render()

    cnt += sim.model.opt.timestep/viewer._time_per_render
    while cnt > 0:
        cnt -= 1
        img = viewer._read_pixels_as_in_window()
        frames.append(img)

        
save_video(frames, 'video/mujoco.mp4', int(0.5/viewer._time_per_render))
show_video('video/mujoco.mp4')

ImportError: To use the imageio ffmpeg plugin you need to 'pip install imageio-ffmpeg'

In [ ]:
save_video(frames, 'video/mujoco.mp4', int(0.5/viewer._time_per_render))
show_video('video/mujoco.mp4')